In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

In [19]:
df_dados = pd.read_excel('dados-2025.xlsx', engine="openpyxl")
df_resultados = pd.read_excel('resultados-2025.xlsx', engine='openpyxl')

eu = df_dados[df_dados['Nome'] == "Matheus Vilarino de Souza Pinto"]
# eu = df_dados[df_dados['Nome'] == "Beatriz dos Santos Marques"]

linha = eu.iloc[0]
m, c, f, N, M = linha['m'], linha['c'], linha['f'], linha['N'], linha['M']
eu

,Nome,ID,m,c,f,N,M
49,Matheus Vilarino de Souza Pinto,50,5.0,0.02,0.55,665,4500



# Questão 1.

## a)

In [20]:
def gerar_tempo_atendimento_tipo1(c_val):
    """
    Gera uma amostra do tempo de atendimento para clientes do Tipo 1.
    Amostra da densidade f(x) = 0,25 + c(x-4) para 2 < x < 6.
    Utiliza o método da amostragem por transformada inversa.
    """
    # Gera um número aleatório uniforme entre 0 e 1
    u = np.random.rand()
    
    # Resolve a equação quadrática F(x) = u, onde F(x) é a CDF.
    # A equação é (c/2)x^2 + (0.25 - 4c)x + (6c - 0.5 - u) = 0
    A = c_val / 2
    B = 0.25 - 4 * c_val
    C = 6 * c_val - 0.5 - u
    
    # Calcula o discriminante, garantindo que não seja negativo por erros de ponto flutuante
    discriminante = B**2 - 4 * A * C
    if discriminante < 0:
        discriminante = 0

    sqrt_discriminante = math.sqrt(discriminante)
    
    # Calcula as duas raízes da equação quadrática
    x1 = (-B + sqrt_discriminante) / (2 * A)
    x2 = (-B - sqrt_discriminante) / (2 * A)
    
    # A raiz correta é a que está no intervalo [2, 6]
    if 2 <= x1 <= 6:
        return x1
    else:
        return x2

In [21]:
def simular_fila(f, c, m, num_clientes=10000):
    """
    Realiza a simulação da fila de atendimento com um único servidor.

    Args:
        f (float): Fração de clientes do Tipo 1 (e.g., 0.7 para 70%).
        c (float): Constante da PDF do Tipo 1.
        m (float): Média de tempo de atendimento para clientes do Tipo 2 (em minutos).
        num_clientes (int): Número de clientes a serem simulados.

    Returns:
        dict: Um dicionário contendo as métricas estatísticas da simulação.
    """
    print(f"Iniciando simulação com f={f}, c={c}, m={m}\n")
    
    # --- 1. Geração dos Tempos de Chegada ---
    # Chegadas seguem um Processo de Poisson com média de 1 a cada 10 min.
    # Isso significa que o tempo entre chegadas segue uma distribuição Exponencial com média 10.
    media_entre_chegadas = 10
    tempos_entre_chegadas = np.random.exponential(scale=media_entre_chegadas, size=num_clientes)
    tempos_de_chegada = np.cumsum(tempos_entre_chegadas)

    # --- 2. Geração dos Tempos de Atendimento ---
    tempos_de_atendimento = []
    # Determina o tipo de cada cliente
    tipos_de_cliente = np.random.rand(num_clientes)
    
    for i in range(num_clientes):
        if tipos_de_cliente[i] < f:
            # Cliente Tipo 1
            tempos_de_atendimento.append(gerar_tempo_atendimento_tipo1(c))
        else:
            # Cliente Tipo 2 (Exponencial com média m)
            tempos_de_atendimento.append(np.random.exponential(scale=m))

    # --- 3. Simulação da Fila ---
    inicios_atendimento = np.zeros(num_clientes)
    fins_atendimento = np.zeros(num_clientes)
    tempos_de_espera = np.zeros(num_clientes)
    tempo_ocioso_total = 0

    # Primeiro cliente
    inicios_atendimento[0] = tempos_de_chegada[0]
    tempos_de_espera[0] = 0
    fins_atendimento[0] = inicios_atendimento[0] + tempos_de_atendimento[0]
    tempo_ocioso_total += tempos_de_chegada[0]  # Ocioso até a primeira chegada

    # Clientes subsequentes
    for i in range(1, num_clientes):
        # O atendente estará livre após o término do atendimento anterior
        atendente_livre_em = fins_atendimento[i-1]
        
        # O atendimento começa no máximo entre a chegada do cliente e a liberação do atendente
        inicios_atendimento[i] = max(tempos_de_chegada[i], atendente_livre_em)
        
        # O tempo de espera é a diferença entre o início do atendimento e a chegada
        tempos_de_espera[i] = inicios_atendimento[i] - tempos_de_chegada[i]
        
        # O atendimento termina após o seu tempo de duração
        fins_atendimento[i] = inicios_atendimento[i] + tempos_de_atendimento[i]

        # Se o cliente chegou depois do atendente ficar livre, houve tempo ocioso
        if tempos_de_chegada[i] > atendente_livre_em:
            tempo_ocioso_total += tempos_de_chegada[i] - atendente_livre_em

    # --- 4. Cálculo dos Resultados ---
    tempo_total_simulacao = fins_atendimento[-1]

    # A) Média e desvio padrão da duração do atendimento
    media_atendimento = np.mean(tempos_de_atendimento)
    dp_atendimento = np.std(tempos_de_atendimento)

    # B) Média e desvio padrão do tempo de espera
    media_espera = np.mean(tempos_de_espera)
    dp_espera = np.std(tempos_de_espera)

    # C) Probabilidade de um cliente esperar mais de 3 minutos
    prob_espera_maior_3 = len(tempos_de_espera[tempos_de_espera > 3]) / num_clientes

    # D) Fração do tempo em que o atendente fica ocioso
    fracao_tempo_ocioso = tempo_ocioso_total / tempo_total_simulacao

    # --- 5. Retorno dos Resultados ---
    resultados = {
        "Duração do Atendimento": {"Média": media_atendimento, "Desvio Padrão": dp_atendimento},
        "Tempo de Espera na Fila": {"Média": media_espera, "Desvio Padrão": dp_espera},
        "Probabilidade de Esperar > 3 min": prob_espera_maior_3,
        "Fração de Tempo Ocioso do Atendente": fracao_tempo_ocioso
    }
    return resultados

In [22]:
# Roda a simulação com as constantes definidas
num_clientes = 10000
estatisticas = simular_fila(f=f, c=c, m=m, num_clientes=num_clientes)

# --- Exibe os resultados formatados ---
print(f"--- Resultados da Simulação ({num_clientes} Clientes) ---")

print("\nA) Duração do Atendimento:")
print(f"   - Média: {estatisticas['Duração do Atendimento']['Média']:.4f} minutos")
print(f"   - Desvio Padrão: {estatisticas['Duração do Atendimento']['Desvio Padrão']:.4f} minutos")

print("\nB) Tempo de Espera na Fila:")
print(f"   - Média: {estatisticas['Tempo de Espera na Fila']['Média']:.4f} minutos")
print(f"   - Desvio Padrão: {estatisticas['Tempo de Espera na Fila']['Desvio Padrão']:.4f} minutos")

print(f"\nC) Probabilidade de Esperar > 3 min: {estatisticas['Probabilidade de Esperar > 3 min']:.2%}")

print(f"\nD) Fração de Tempo Ocioso do Atendente: {estatisticas['Fração de Tempo Ocioso do Atendente']:.2%}")

Iniciando simulação com f=0.55, c=0.02, m=5.0

--- Resultados da Simulação (10000 Clientes) ---

A) Duração do Atendimento:
   - Média: 4.5385 minutos
   - Desvio Padrão: 3.4129 minutos

B) Tempo de Espera na Fila:
   - Média: 3.0298 minutos
   - Desvio Padrão: 5.6158 minutos

C) Probabilidade de Esperar > 3 min: 29.21%

D) Fração de Tempo Ocioso do Atendente: 53.97%


## b)

In [23]:
E = f * (4 + (16 / 3) * c) + (1 - f) * m
print(f"Esperança real: {E}")

Esperança real: 4.508666666666667


In [24]:
desvio_padrao = math.sqrt(((12 - 256 * c ** 2) / 9) * f + m ** 2 * (1 - f) + f * (1 - f) * (4 + (16 / 3) * c - m) ** 2)

print(f"Desvio Padrão real: {desvio_padrao}")

Desvio Padrão real: 3.4892107353319255


# Questão 2.

In [25]:
def simular_figurinhas(N : int) -> int:
    """
    Realiza uma simulação de álbum de figurinhas.

    Args:
        N (int): Total de figurinhas para completar no álbum.

    Returns:
        int: Total de cartinhas compradas.
    """
    figurinhas_obtidas = set()
    num_compras = 0

    while len(figurinhas_obtidas) < N:
        num_compras += 1

        nova_figurinha = np.random.randint(N)
        figurinhas_obtidas.add(nova_figurinha)

    return num_compras

In [26]:
num_simulacoes_figurinhas = 1000
iteracoes_simulacoes = []
for _ in range(num_simulacoes_figurinhas):
    iteracoes_simulacoes.append(simular_figurinhas(N))

iteracoes_simulacoes = np.array(iteracoes_simulacoes)

In [27]:
print(f"--- Resultados da Simulação ({num_simulacoes_figurinhas} álbuns) ---")

print("\nA) Total de Figurinhas Compradas:")
print(f"   - Média: {iteracoes_simulacoes.mean():.4f} figurinhas")
print(f"   - Desvio Padrão: {iteracoes_simulacoes.std():.4f} figurinhas")

print("\nC) Total Acumulado:")
print(f"   - Probabilidade de Comprar > {M}: {len(iteracoes_simulacoes[iteracoes_simulacoes > M]) / num_simulacoes_figurinhas }")

--- Resultados da Simulação (1000 álbuns) ---

A) Total de Figurinhas Compradas:
   - Média: 4682.8950 figurinhas
   - Desvio Padrão: 828.0099 figurinhas

C) Total Acumulado:
   - Probabilidade de Comprar > 4500: 0.537


In [28]:
E = 0
for i in range(1, N + 1):
    E += N / i

print(f"Esperança real: {E}")

Esperança real: 4706.706673882401


In [29]:
var = 0
for i in range(1, N + 1):
    var += N ** 2 / (i ** 2) - N / i
    
desvio_padrao = math.sqrt(var)

print(f"Desvio padrão real: {desvio_padrao}")

Desvio padrão real: 849.7409962967828


In [30]:
def simular_figurinhas_com_2_albuns(N: int):
    """
    Realiza uma simulação de compra de figurinhas com dois álbuns de tamanho N.

    Args:
        N (int): Total de figurinhas distintas para completar um álbum.

    Returns:
        int: Total de figurinhas compradas até que ambos os álbuns estejam completos.
    """
    
    figurinhas_obtidas_1 = set()
    figurinhas_obtidas_2 = set()
    num_compras = 0

    while len(figurinhas_obtidas_1) + len(figurinhas_obtidas_2) < 2 * N:
        num_compras += 1
        nova_figurinha = np.random.randint(N)
        if nova_figurinha not in figurinhas_obtidas_1:
            figurinhas_obtidas_1.add(nova_figurinha)
            continue
        
        figurinhas_obtidas_2.add(nova_figurinha)

    return num_compras

In [31]:
iteracoes_simulacoes_2 = []
for _ in range(num_simulacoes_figurinhas):
    total = simular_figurinhas_com_2_albuns(N=N)
    iteracoes_simulacoes_2.append(total)
iteracoes_simulacoes_2 = np.array(iteracoes_simulacoes_2)


In [32]:

print(f"Média de figurinhas para dois álbuns: {iteracoes_simulacoes_2.mean()}")
print(f"Média de figurinhas por álbum: {iteracoes_simulacoes_2.mean() / 2}")

Média de figurinhas para dois álbuns: 6260.853
Média de figurinhas por álbum: 3130.4265
